# Demo NOAA model file aggregation

This code is from `LibGOODS/model_catalogs` which is currently in a private repository for NOAA but later will be public. And actually the version of the code I have here is an older version that is easier to separate from the rest of the code base.

In [1]:
import pandas as pd
from siphon.catalog import TDSCatalog
import fnmatch
import re
import xarray as xr

## Function

In [2]:
def agg_for_date(
    date, catloc, filetype, treat_last_day_as_forecast=False, pattern=None
):
    """Aggregate NOAA OFS-style nowcast/forecast files.
    Parameters
    ----------
    date: str of datetime, pd.Timestamp
        Date of day to find model output files for. Doesn't pay attention to hours/minutes/seconds.
    catloc: str
        URL of thredds base catalog. Should be a pattern like the following in which the date will be
        filled in as a f-string variable. Can be xml or html:
        https://opendap.co-ops.nos.noaa.gov/thredds/catalog/NOAA/CBOFS/MODELS/{date.year}/{str(date.month).zfill(2)}/{str(date.day).zfill(2)}/catalog.xml
    filetype: str
        Which filetype to use. Every NOAA OFS model has "fields" available, but some have "regulargrid"
        or "2ds" also. This availability information is in the source catalog for the model under
        `filetypes` metadata.
    treat_last_day_as_forecast: bool, optional
        If True, then date is the last day of the time period being sought and the forecast files
        should be brought in along with the nowcast files, to get the model output the length of the
        forecast out in time. The forecast files brought in will have the latest timing cycle of the
        day that is available. If False, all nowcast files (for all timing cycles) are brought in.
    pattern: str, optional
        If a model file pattern doesn't match that assumed in this code, input one that will work.
        Currently only NYOFS doesn't match but the pattern is built into the catalog file.
    Returns
    -------
    List of URLs for where to find all of the model output files that match the keyword arguments.
    """

    if not isinstance(date, pd.Timestamp):
        date = pd.Timestamp(date)

    # f format catloc name with input date
    catloc = eval(f"f'{catloc}'")
    cat = TDSCatalog(catloc)

    # # brings in nowcast and forecast for any date in the catalog
    # pattern0 = f'*{filetype}*.t??z.*'
    # # brings in nowcast and forecast for only the day specified
    # pattern0 = date.strftime(f'*{filetype}*.%Y%m%d.t??z.*')

    if pattern is None:
        pattern = date.strftime(f"*{filetype}*.n*.%Y%m%d.t??z.*")
        # pattern = date.strftime(f'*{filetype}*.n???.%Y%m%d.t??z.*')
    else:
        pattern = eval(f"f'{pattern}'")
    # pattern = date.strftime(f'*{filetype}*.n*.%Y%m%d.t??z.*')

    # '*{filetype}.n???.{date.year}{str(date.month).zfill(2)}{str(date.day).zfill(2)}.t??z.*'

    # pattern = eval(f"f'{pattern}'")
    # import pdb; pdb.set_trace()
    fnames = fnmatch.filter(cat.datasets, pattern)

    if treat_last_day_as_forecast:

        import re

        regex = re.compile(".t[0-9]{2}z.")
        cycle = sorted(  # noqa: F841
            list(set([substr[2:4] for substr in regex.findall("".join(cat.datasets))]))
        )[
            -1
        ]  # noqa: E501

        # cycle = sorted(list(set([fname[ fname.find(start:='.t') + len(start):fname.find('z.')] for fname in fnames])))[-1]  # noqa: E501
        # import pdb; pdb.set_trace()
        # pattern1 = f'*{filetype}*.t{cycle}z.*'

        # replace '.t??z.' in pattern with '.t{cycle}z.' and replace '.n*.' with '.*.'
        pattern1 = pattern.replace(".t??z.", ".t{cycle}z.").replace(".n*.", ".*.")
        pattern1 = eval(f"f'{pattern1}'")
        fnames = fnmatch.filter(cat.datasets, pattern1)

    filelocs = [cat.datasets.get(fname).access_urls["OPENDAP"] for fname in fnames]

    return filelocs


## Example

### Run aggregation code for only nowcast files

In [6]:
date = pd.Timestamp.today()

# This is a base THREDDS catalog for one model: CBOFS
catloc = date.strftime('https://opendap.co-ops.nos.noaa.gov/thredds/catalog/NOAA/CBOFS/MODELS/%Y/%m/%d/catalog.xml')

filetype = 'fields'

# Set to True to make coherent series of nowcast followed by forecast files
# False just pulls out nowcast files
# Which "timing" cycle to use changes depending on this choice too
treat_last_day_as_forecast = False

# get the file locations
filelocs = agg_for_date(date, catloc, filetype, treat_last_day_as_forecast=treat_last_day_as_forecast)

# read in with xarray, which will put them in order and remove duplicates
ds = xr.open_mfdataset(filelocs)
ds

<xarray.Dataset>
Dimensions:         (ocean_time: 18, tracer: 3, boundary: 4, s_rho: 20,
                     s_w: 21, eta_rho: 291, xi_rho: 332, eta_u: 291, xi_u: 331,
                     eta_v: 290, xi_v: 332, eta_psi: 290, xi_psi: 331)
Coordinates:
  * s_rho           (s_rho) float64 -0.975 -0.925 -0.875 ... -0.075 -0.025
  * s_w             (s_w) float64 -1.0 -0.95 -0.9 -0.85 ... -0.15 -0.1 -0.05 0.0
    lon_rho         (eta_rho, xi_rho) float64 dask.array<chunksize=(291, 332), meta=np.ndarray>
    lat_rho         (eta_rho, xi_rho) float64 dask.array<chunksize=(291, 332), meta=np.ndarray>
    lon_u           (eta_u, xi_u) float64 dask.array<chunksize=(291, 331), meta=np.ndarray>
    lat_u           (eta_u, xi_u) float64 dask.array<chunksize=(291, 331), meta=np.ndarray>
    lon_v           (eta_v, xi_v) float64 dask.array<chunksize=(290, 332), meta=np.ndarray>
    lat_v           (eta_v, xi_v) float64 dask.array<chunksize=(290, 332), meta=np.ndarray>
    lon_psi         (eta_psi, xi_psi) float64 dask.array<chunksize=(290, 331), meta=np.ndarray>
    lat_psi         (eta_psi, xi_psi) float64 dask.array<chunksize=(290, 331), meta=np.ndarray>
  * ocean_time      (ocean_time) datetime64[ns] 2022-04-07T19:00:00 ... 2022-...
Dimensions without coordinates: tracer, boundary, eta_rho, xi_rho, eta_u, xi_u,
                                eta_v, xi_v, eta_psi, xi_psi
Data variables: (12/97)
    ntimes          (ocean_time) int32 4320 4320 4320 4320 ... 4320 4320 4320
    ndtfast         (ocean_time) int32 20 20 20 20 20 20 ... 20 20 20 20 20 20
    dt              (ocean_time) float64 5.0 5.0 5.0 5.0 5.0 ... 5.0 5.0 5.0 5.0
    dtfast          (ocean_time) float64 0.25 0.25 0.25 0.25 ... 0.25 0.25 0.25
    dstart          (ocean_time) datetime64[ns] 2022-04-07T18:00:00 ... 2022-...
    nHIS            (ocean_time) int32 720 720 720 720 720 ... 720 720 720 720
    ...              ...
    temp            (ocean_time, s_rho, eta_rho, xi_rho) float32 dask.array<chunksize=(1, 20, 291, 332), meta=np.ndarray>
    salt            (ocean_time, s_rho, eta_rho, xi_rho) float32 dask.array<chunksize=(1, 20, 291, 332), meta=np.ndarray>
    oxygen          (ocean_time, s_rho, eta_rho, xi_rho) float32 dask.array<chunksize=(1, 20, 291, 332), meta=np.ndarray>
    Pair            (ocean_time, eta_rho, xi_rho) float32 dask.array<chunksize=(1, 291, 332), meta=np.ndarray>
    Uwind           (ocean_time, eta_rho, xi_rho) float32 dask.array<chunksize=(1, 291, 332), meta=np.ndarray>
    Vwind           (ocean_time, eta_rho, xi_rho) float32 dask.array<chunksize=(1, 291, 332), meta=np.ndarray>
Attributes: (12/37)
    file:                            nos.cbofs.fields.nowcast.20220408.t00z_0...
    format:                          netCDF-4/HDF5 file
    Conventions:                     CF-1.4, SGRID-0.3
    type:                            ROMS/TOMS history file
    title:                           cbofs nowcast RUN in operational mode
    var_info:                        varinfo.dat
    ...                              ...
    history:                         ROMS/TOMS, Version 3.9, Friday - April 8...
    ana_file:                        ROMS/Functionals/ana_btflux.h, ROMS/Func...
    CPP_options:                     mode, ADD_FSOBC, ADD_M2OBC, ANA_BPFLUX, ...
    bio_file:                        ROMS/Nonlinear/Biology/hypoxia_srm.h
    DODS_EXTRA.Unlimited_Dimension:  ocean_time
    EXTRA_DIMENSION.N:               20

In [7]:
filelocs

['https://opendap.co-ops.nos.noaa.gov/thredds/dodsC/NOAA/CBOFS/MODELS/2022/04/08/nos.cbofs.fields.n006.20220408.t12z.nc',
 'https://opendap.co-ops.nos.noaa.gov/thredds/dodsC/NOAA/CBOFS/MODELS/2022/04/08/nos.cbofs.fields.n006.20220408.t06z.nc',
 'https://opendap.co-ops.nos.noaa.gov/thredds/dodsC/NOAA/CBOFS/MODELS/2022/04/08/nos.cbofs.fields.n006.20220408.t00z.nc',
 'https://opendap.co-ops.nos.noaa.gov/thredds/dodsC/NOAA/CBOFS/MODELS/2022/04/08/nos.cbofs.fields.n005.20220408.t12z.nc',
 'https://opendap.co-ops.nos.noaa.gov/thredds/dodsC/NOAA/CBOFS/MODELS/2022/04/08/nos.cbofs.fields.n005.20220408.t06z.nc',
 'https://opendap.co-ops.nos.noaa.gov/thredds/dodsC/NOAA/CBOFS/MODELS/2022/04/08/nos.cbofs.fields.n005.20220408.t00z.nc',
 'https://opendap.co-ops.nos.noaa.gov/thredds/dodsC/NOAA/CBOFS/MODELS/2022/04/08/nos.cbofs.fields.n004.20220408.t12z.nc',
 'https://opendap.co-ops.nos.noaa.gov/thredds/dodsC/NOAA/CBOFS/MODELS/2022/04/08/nos.cbofs.fields.n004.20220408.t06z.nc',
 'https://opendap.co-ops

### Run aggregation code for nowcast + forecast

In [9]:
# Set to True to make coherent series of nowcast followed by forecast files
# False just pulls out nowcast files
# Which "timing" cycle to use changes depending on this choice too
treat_last_day_as_forecast = True

# get the file locations
filelocs = agg_for_date(date, catloc, filetype, treat_last_day_as_forecast=treat_last_day_as_forecast)

# read in with xarray, which will put them in order and remove duplicates
ds = xr.open_mfdataset(filelocs)
ds

<xarray.Dataset>
Dimensions:         (ocean_time: 54, tracer: 3, boundary: 4, s_rho: 20,
                     s_w: 21, eta_rho: 291, xi_rho: 332, eta_u: 291, xi_u: 331,
                     eta_v: 290, xi_v: 332, eta_psi: 290, xi_psi: 331)
Coordinates:
  * s_rho           (s_rho) float64 -0.975 -0.925 -0.875 ... -0.075 -0.025
  * s_w             (s_w) float64 -1.0 -0.95 -0.9 -0.85 ... -0.15 -0.1 -0.05 0.0
    lon_rho         (eta_rho, xi_rho) float64 dask.array<chunksize=(291, 332), meta=np.ndarray>
    lat_rho         (eta_rho, xi_rho) float64 dask.array<chunksize=(291, 332), meta=np.ndarray>
    lon_u           (eta_u, xi_u) float64 dask.array<chunksize=(291, 331), meta=np.ndarray>
    lat_u           (eta_u, xi_u) float64 dask.array<chunksize=(291, 331), meta=np.ndarray>
    lon_v           (eta_v, xi_v) float64 dask.array<chunksize=(290, 332), meta=np.ndarray>
    lat_v           (eta_v, xi_v) float64 dask.array<chunksize=(290, 332), meta=np.ndarray>
    lon_psi         (eta_psi, xi_psi) float64 dask.array<chunksize=(290, 331), meta=np.ndarray>
    lat_psi         (eta_psi, xi_psi) float64 dask.array<chunksize=(290, 331), meta=np.ndarray>
  * ocean_time      (ocean_time) datetime64[ns] 2022-04-08T07:00:00 ... 2022-...
Dimensions without coordinates: tracer, boundary, eta_rho, xi_rho, eta_u, xi_u,
                                eta_v, xi_v, eta_psi, xi_psi
Data variables: (12/97)
    ntimes          (ocean_time) int32 4320 4320 4320 4320 ... 34560 34560 34560
    ndtfast         (ocean_time) int32 20 20 20 20 20 20 ... 20 20 20 20 20 20
    dt              (ocean_time) float64 5.0 5.0 5.0 5.0 5.0 ... 5.0 5.0 5.0 5.0
    dtfast          (ocean_time) float64 0.25 0.25 0.25 0.25 ... 0.25 0.25 0.25
    dstart          (ocean_time) datetime64[ns] 2022-04-08T06:00:00 ... 2022-...
    nHIS            (ocean_time) int32 720 720 720 720 720 ... 720 720 720 720
    ...              ...
    temp            (ocean_time, s_rho, eta_rho, xi_rho) float32 dask.array<chunksize=(1, 20, 291, 332), meta=np.ndarray>
    salt            (ocean_time, s_rho, eta_rho, xi_rho) float32 dask.array<chunksize=(1, 20, 291, 332), meta=np.ndarray>
    oxygen          (ocean_time, s_rho, eta_rho, xi_rho) float32 dask.array<chunksize=(1, 20, 291, 332), meta=np.ndarray>
    Pair            (ocean_time, eta_rho, xi_rho) float32 dask.array<chunksize=(1, 291, 332), meta=np.ndarray>
    Uwind           (ocean_time, eta_rho, xi_rho) float32 dask.array<chunksize=(1, 291, 332), meta=np.ndarray>
    Vwind           (ocean_time, eta_rho, xi_rho) float32 dask.array<chunksize=(1, 291, 332), meta=np.ndarray>
Attributes: (12/37)
    file:                            nos.cbofs.fields.nowcast.20220408.t12z_0...
    format:                          netCDF-4/HDF5 file
    Conventions:                     CF-1.4, SGRID-0.3
    type:                            ROMS/TOMS history file
    title:                           cbofs nowcast RUN in operational mode
    var_info:                        varinfo.dat
    ...                              ...
    history:                         ROMS/TOMS, Version 3.9, Friday - April 8...
    ana_file:                        ROMS/Functionals/ana_btflux.h, ROMS/Func...
    CPP_options:                     mode, ADD_FSOBC, ADD_M2OBC, ANA_BPFLUX, ...
    bio_file:                        ROMS/Nonlinear/Biology/hypoxia_srm.h
    DODS_EXTRA.Unlimited_Dimension:  ocean_time
    EXTRA_DIMENSION.N:               20

In [10]:
filelocs

['https://opendap.co-ops.nos.noaa.gov/thredds/dodsC/NOAA/CBOFS/MODELS/2022/04/08/nos.cbofs.fields.n006.20220408.t12z.nc',
 'https://opendap.co-ops.nos.noaa.gov/thredds/dodsC/NOAA/CBOFS/MODELS/2022/04/08/nos.cbofs.fields.n005.20220408.t12z.nc',
 'https://opendap.co-ops.nos.noaa.gov/thredds/dodsC/NOAA/CBOFS/MODELS/2022/04/08/nos.cbofs.fields.n004.20220408.t12z.nc',
 'https://opendap.co-ops.nos.noaa.gov/thredds/dodsC/NOAA/CBOFS/MODELS/2022/04/08/nos.cbofs.fields.n003.20220408.t12z.nc',
 'https://opendap.co-ops.nos.noaa.gov/thredds/dodsC/NOAA/CBOFS/MODELS/2022/04/08/nos.cbofs.fields.n002.20220408.t12z.nc',
 'https://opendap.co-ops.nos.noaa.gov/thredds/dodsC/NOAA/CBOFS/MODELS/2022/04/08/nos.cbofs.fields.n001.20220408.t12z.nc',
 'https://opendap.co-ops.nos.noaa.gov/thredds/dodsC/NOAA/CBOFS/MODELS/2022/04/08/nos.cbofs.fields.f048.20220408.t12z.nc',
 'https://opendap.co-ops.nos.noaa.gov/thredds/dodsC/NOAA/CBOFS/MODELS/2022/04/08/nos.cbofs.fields.f047.20220408.t12z.nc',
 'https://opendap.co-ops